In [1]:
import pandas as pd
from pandas_datareader import data
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests


In [2]:
def getTicker(company_name):
    url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={company_name}&apikey=1111111111'
    response = requests.get(url=url)
    data = response.json()
    return [x.get('1. symbol') for x in data.get('bestMatches', [])]


In [3]:
tickers = ['AAPL', 'AMZN']

start_date = (datetime.now() - relativedelta(years=5)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Use pandas_reader.data.DataReader to load data
panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)


In [4]:
moving_averages = [50, 100, 200]
for tic in tickers:
    for ma in moving_averages:
        panel_data[f'ma{ma}', tic] = panel_data.Close[tic].rolling(
            window=ma).mean()


In [5]:
getTicker('tesla')

['TSLA', 'TL0.DEX', 'TL0.FRK', 'TSLA34.SAO']